# 📘 LLMentor – Notebook per gestione Syllabus PDF

Questo notebook ti permette di:
- Caricare un file PDF di syllabus
- Estrarne il testo
- Convertirlo in formato tabellare
- Salvare il risultato in un file CSV nella cartella `data/`


In [6]:
import pandas as pd
import fitz  # PyMuPDF
import re
import os
from collections import defaultdict

In [7]:
def estrai_testo_da_pdf(percorso_pdf):
    """Estrae il testo da un file PDF."""
    try:
        doc = fitz.open(percorso_pdf)
        testo_completo = ""
        for page in doc:
            testo_completo += page.get_text()
        return testo_completo
    except Exception as e:
        print(f"Errore nell'apertura o estrazione del PDF {percorso_pdf}: {str(e)}")
        return ""

In [8]:
def analizza_struttura_pdf(testo):
    """Analizza la struttura del testo per identificare pattern di lezioni o argomenti."""
    print("\n=== ANALISI DIAGNOSTICA DEL PDF ===")

    # Visualizza le prime 15 righe non vuote per capire la struttura
    print("Prime 15 righe di contenuto:")
    lines = [line.strip() for line in testo.splitlines() if line.strip()]
    for i, line in enumerate(lines[:15]):
        print(f"{i+1}. {line}")

    # Cerca pattern comuni in syllabus
    patterns = {
        "settimana": r"(?:settimana|week)\s+(\d+)",
        "lezione": r"(?:lezione|lecture)\s+(\d+)",
        "sessione": r"(?:sessione|session)\s+(\d+)",
        "modulo": r"(?:modulo|module)\s+(\d+)",
        "argomento_num": r"^(\d+)[\.\s\-:]",
        "argomento_lettera": r"^([a-z])[\.\s\-:]",
        "contenuto": r"(?:contenuto|contenuti|content|contents|programma|program|syllabus)"
    }

    pattern_trovati = {}
    for nome, pattern in patterns.items():
        matches = re.findall(pattern, testo, re.IGNORECASE | re.MULTILINE)
        pattern_trovati[nome] = len(matches)
        if matches:
            print(f"Pattern '{nome}' trovato {len(matches)} volte. Esempi: {matches[:3]}")

    # Verifica se ci sono tabelle
    tabelle_potenziali = len(re.findall(r"\|\s*\w+\s*\|", testo))
    if tabelle_potenziali > 0:
        print(f"Possibile presenza di tabelle: {tabelle_potenziali} istanze di pattern '|'")

    print("====================================")

    return pattern_trovati

def estrai_blocchi_testo(testo):
    """Estrae blocchi di testo che probabilmente contengono argomenti del corso."""
    # Elimina interruzioni di riga eccessive
    testo = re.sub(r'\n{3,}', '\n\n', testo)

    # Cerca blocchi che sembrano contenere il programma del corso
    blocchi_programma = []
    linee = testo.splitlines()
    blocco_corrente = []
    in_blocco_programma = False

    trigger_words = [
        "programma", "syllabus", "contenuti", "contents", "argomenti",
        "lezioni", "lectures", "calendario", "schedule", "settimane",
        "weeks", "moduli", "modules", "teaching", "didattica"
    ]

    for i, line in enumerate(linee):
        line_lower = line.lower().strip()

        # Controlla se è l'inizio di un blocco di programma
        if not in_blocco_programma:
            for word in trigger_words:
                if word in line_lower and len(line.strip()) < 100:  # Evita testi troppo lunghi
                    in_blocco_programma = True
                    blocco_corrente = []
                    print(f"Possibile blocco programma trovato alla riga {i+1}: {line.strip()}")
                    break

        # Se siamo in un blocco programma, aggiungi la riga
        if in_blocco_programma:
            # Controllo se è la fine del blocco (sezione nuova)
            end_triggers = ["bibliografia", "reference", "valutazione", "assessment",
                          "esame", "exam", "prerequisiti", "requirements", "materiale didattico"]

            if any(trigger in line_lower for trigger in end_triggers) and len(line.strip()) < 100:
                if blocco_corrente:
                    blocchi_programma.append('\n'.join(blocco_corrente))
                    print(f"Fine blocco programma alla riga {i+1}: {line.strip()}")
                in_blocco_programma = False
            else:
                blocco_corrente.append(line)

    # Aggiungi l'ultimo blocco se presente
    if in_blocco_programma and blocco_corrente:
        blocchi_programma.append('\n'.join(blocco_corrente))

    if not blocchi_programma:
        print("Nessun blocco di programma identificato chiaramente. Provo con tutto il testo.")
        return [testo]

    print(f"Trovati {len(blocchi_programma)} blocchi di programma potenziali.")
    return blocchi_programma


In [9]:
def identifica_pattern_lezioni(testo):
    """Identifica i pattern più probabili per le lezioni/argomenti nel syllabus."""
    # Pattern comuni per numeri di lezione/argomento
    patterns = [
        (r"(?:lezione|lecture)\s+(\d+)[\s:\.\-]+(.+?)(?=\n|$)", "Lezione"),
        (r"(?:settimana|week)\s+(\d+)[\s:\.\-]+(.+?)(?=\n|$)", "Settimana"),
        (r"^(?:session|sessione)\s+(\d+)[\s:\.\-]+(.+?)(?=\n|$)", "Sessione"),
        (r"^(?:modulo|module)\s+(\d+)[\s:\.\-]+(.+?)(?=\n|$)", "Modulo"),
        (r"^(\d+)[\.\s\-:]+(.+?)(?=\n|$)", "Argomento"),
        (r"^([A-Z][\.\)])[\s\-:]+(.+?)(?=\n|$)", "Argomento Lettera")
    ]

    results = []
    for pattern, label in patterns:
        matches = re.findall(pattern, testo, re.IGNORECASE | re.MULTILINE)
        if matches:
            print(f"Pattern {label} trovato {len(matches)} volte.")
            results.append((pattern, label, len(matches)))

    # Ritorna il pattern con più match
    if results:
        return sorted(results, key=lambda x: x[2], reverse=True)[0]
    else:
        return None

In [10]:
def parser_syllabus_adattivo(testo, nome_file=""):
    """Parser che si adatta alla struttura del syllabus identificata."""
    print("\n=== INIZIO PARSING ADATTIVO ===")

    # Analizziamo la struttura
    pattern_trovati = analizza_struttura_pdf(testo)

    # Estrai i blocchi di testo che probabilmente contengono il programma
    blocchi_programma = estrai_blocchi_testo(testo)

    # Unisci i blocchi in un unico testo per l'analisi
    testo_programma = "\n\n".join(blocchi_programma)

    # Identifica il pattern più probabile per le lezioni
    pattern_info = identifica_pattern_lezioni(testo_programma)

    if not pattern_info:
        print("Nessun pattern adatto identificato nel testo.")
        return pd.DataFrame()

    pattern, label, _ = pattern_info
    print(f"Utilizzo pattern '{label}' per l'estrazione: {pattern}")

    # Estrai lezioni/argomenti
    risultati = []
    matches = re.findall(pattern, testo_programma, re.IGNORECASE | re.MULTILINE)

    if not matches:
        print("Nessuna corrispondenza trovata con il pattern selezionato.")
        return pd.DataFrame()

    for num, contenuto in matches:
        num = num.strip()
        contenuto = contenuto.strip()

        # Cerca di estrarre sotto-argomenti
        sotto_argomenti = []
        linee = contenuto.split("\n")
        argomento_principale = linee[0].strip() if linee else contenuto

        # Identifica sotto-argomenti dalle linee successive
        for linea in linee[1:]:
            linea = linea.strip()
            if linea and not re.search(r"^(settimana|lezione|week|lecture|sessione|session|modulo|module)\s+\d+", linea, re.IGNORECASE):
                # Rimuovi eventuali bullet o numeri all'inizio
                linea = re.sub(r"^[\-•\*\d\.\s]+", "", linea).strip()
                if linea:
                    sotto_argomenti.append(linea)

        dettagli = " | ".join(sotto_argomenti)

        risultati.append({
            "numero": int(num) if num.isdigit() else num,
            "tipo": label,
            "argomento_principale": argomento_principale,
            "dettagli": dettagli,
            "file_origine": nome_file
        })

    print(f"Estratti {len(risultati)} argomenti/lezioni.")

    # Crea e restituisci il DataFrame
    df = pd.DataFrame(risultati)

    # Rinomina le colonne in base al tipo di pattern trovato
    df = df.rename(columns={"numero": label.lower()})

    return df

In [11]:
def cerca_alternative(testo):
    """Cerca pattern alternativi se i metodi standard falliscono."""
    print("\n=== RICERCA PATTERN ALTERNATIVI ===")

    risultati = []

    # Pattern alternativo 1: Elenchi puntati/numerati con titoli in maiuscolo
    pattern_titoli = r"^[•\-\*]?\s*([A-Z][A-Z\s]+[A-Z])[\s:\.\-]*$(.*?)(?=^[•\-\*]?\s*[A-Z][A-Z\s]+[A-Z][\s:\.\-]*$|$)"
    titoli = re.findall(pattern_titoli, testo, re.MULTILINE | re.DOTALL)

    if titoli:
        print(f"Trovati {len(titoli)} possibili titoli di sezione in maiuscolo.")
        for i, (titolo, contenuto) in enumerate(titoli):
            titolo = titolo.strip()
            contenuto = contenuto.strip()

            # Estrai potenziali sotto-argomenti (frasi o punti elenco)
            sotto_argomenti = []
            for linea in contenuto.split("\n"):
                linea = linea.strip()
                if linea and len(linea) > 10:  # ignora linee troppo corte
                    # Rimuovi eventuali bullet o numeri all'inizio
                    linea = re.sub(r"^[\-•\*\d\.\s]+", "", linea).strip()
                    if linea:
                        sotto_argomenti.append(linea)

            dettagli = " | ".join(sotto_argomenti)

            risultati.append({
                "sezione": i + 1,
                "tipo": "Sezione",
                "argomento_principale": titolo,
                "dettagli": dettagli,
                "file_origine": ""
            })

    # Pattern alternativo 2: Paragrafi con potenziali argomenti
    if not risultati:
        # Cerca paragrafi che sembrano argomenti (primo termine in grassetto o prima riga corta seguita da testo)
        paragrafi = re.split(r"\n\s*\n", testo)
        count = 0
        for i, paragrafo in enumerate(paragrafi):
            paragrafo = paragrafo.strip()
            if paragrafo and len(paragrafo) > 30:  # ignora paragrafi troppo corti
                linee = paragrafo.splitlines()
                if linee and len(linee[0]) < 80:  # la prima linea è potenzialmente un titolo
                    titolo = linee[0].strip()
                    contenuto = "\n".join(linee[1:]).strip()

                    # Solo se sembra un potenziale argomento e non informazioni amministrative
                    if not re.search(r"(mail|tel|ricevimento|exam|esame|bibliografia|valutazione)", titolo, re.IGNORECASE):
                        count += 1
                        risultati.append({
                            "paragrafo": count,
                            "tipo": "Paragrafo",
                            "argomento_principale": titolo,
                            "dettagli": contenuto,
                            "file_origine": ""
                        })

        if count > 0:
            print(f"Estratti {count} potenziali paragrafi con argomenti.")

    return pd.DataFrame(risultati) if risultati else None

def processa_pdf_adattivo(percorso_pdf):
    """Processa un PDF con approccio adattivo alla struttura."""
    try:
        nome_file = os.path.basename(percorso_pdf)
        print(f"\nElaborazione file: {percorso_pdf}")

        # Estrai testo
        testo = estrai_testo_da_pdf(percorso_pdf)
        if not testo:
            print("Nessun testo estratto dal PDF.")
            return pd.DataFrame()

        # Prova il parser adattivo
        df = parser_syllabus_adattivo(testo, nome_file)

        # Se non otteniamo risultati, prova metodi alternativi
        if df.empty:
            print("Parser adattivo non ha trovato contenuti. Provo approcci alternativi...")
            df_alt = cerca_alternative(testo)
            if df_alt is not None and not df_alt.empty:
                df = df_alt
                df['file_origine'] = nome_file

        # Se ancora vuoto, analizza un'ultima volta il testo riga per riga
        if df.empty:
            print("Tutti i metodi standard falliti. Eseguendo analisi di fallback...")
            linee = [line.strip() for line in testo.splitlines() if line.strip() and len(line.strip()) > 10]

            # Cerca linee che sembrano avere contenuto formativo (evitando info amministrative)
            exclude_pattern = r"(bibliografia|docente|ricevimento|esame|valutazione|prerequisiti|e-mail|tel|www)"
            argomenti = []
            for i, linea in enumerate(linee):
                if not re.search(exclude_pattern, linea, re.IGNORECASE) and len(linea) < 200:
                    argomenti.append({
                        "linea": i + 1,
                        "tipo": "Contenuto",
                        "argomento_principale": linea,
                        "dettagli": "",
                        "file_origine": nome_file
                    })

            if argomenti:
                print(f"Analisi di fallback: estratte {len(argomenti)} potenziali linee di contenuto.")
                df = pd.DataFrame(argomenti)

        return df

    except Exception as e:
        print(f"Errore nell'elaborazione del PDF: {str(e)}")
        return pd.DataFrame()

In [12]:
def salva_output(df, percorso_pdf):
    """Salva il DataFrame in CSV con gestione del nome file."""
    if df.empty:
        print("Nessun dato da salvare.")
        return

    nome_base = os.path.splitext(os.path.basename(percorso_pdf))[0]
    os.makedirs("data", exist_ok=True)
    output_path = f"data/{nome_base}.csv"

    df.to_csv(output_path, index=False)
    print(f"File CSV salvato in: {output_path}")

    # Mostra esempio
    print("\nEsempio contenuto CSV:")
    print(df.head())
    print(f"Totale righe: {len(df)}")

def main():
    """Funzione principale."""
    # Controlla se viene fornito un percorso file come argomento
    if len(sys.argv) > 1:
        percorso_pdf = sys.argv[1]
    else:
        percorso_pdf = input("Inserisci il percorso del file PDF: ")

    # Processa il file
    df = processa_pdf_adattivo(percorso_pdf)

    # Salva i risultati
    salva_output(df, percorso_pdf)

if __name__ == "__main__":
    main()


Elaborazione file: -f
Errore nell'apertura o estrazione del PDF -f: no such file: '-f'
Nessun testo estratto dal PDF.
Nessun dato da salvare.
